# Homework #2



## Question #1

There are four text files in the data folder

* Atatürk's "Nutuk" in Turkish
* Dicken's novel "Great Expectations" in English
* Flauberts' novel "Madam Bovary" in French
* A text file `unknown.txt` in an unknown language

Your tasks are

* Calculate how many times each character (letter) appear in each text.
* Calculate the character distributions, i.e. using the character counts, calculate the probability of each character appearing in the text.
* Find the set of characters common to all three texts.
* Using the common set and the KL-divergence, show that each language have different character distributions.
* Determine the language of the text file `unknown.txt` KL-divergence measure.

In [6]:
import pandas as pd
import numpy as np
import string
from re import sub
from collections import Counter

alltexts=["ataturk_nutuk.txt","dickens_great_expectations.txt","flaubert_madame_bovary.txt","unknown.txt"]
texts=[]

def readtext(name):
    with open(name, encoding='utf-8') as text:
        x=text.read().lower()
    res=list(sub(r'[\W\d ]+', '', x))
    return res
for text in alltexts:
    temp=readtext(text)
    texts.append(temp)
    
count= [Counter(text) for text in texts] 



In [26]:
count

[Counter({'y': 29496,
          'u': 39565,
          'r': 74152,
          'd': 45861,
          'm': 70652,
          'z': 15593,
          'n': 78256,
          'p': 7371,
          'a': 118982,
          'ç': 8613,
          'l': 75010,
          'ı': 44210,
          'i': 100889,
          'ş': 19525,
          'g': 14552,
          'e': 113678,
          'ğ': 10530,
          'ü': 19746,
          'b': 29389,
          'k': 48264,
          't': 42730,
          'h': 12834,
          'ö': 7358,
          'o': 20290,
          's': 30794,
          'â': 2643,
          'v': 15618,
          'c': 10929,
          'î': 1469,
          'f': 7489,
          'j': 163,
          'á': 2,
          'ì': 5,
          'û': 836,
          'ó': 1,
          'w': 66,
          'í': 4,
          'ú': 1,
          '_': 42,
          'q': 1,
          'é': 30,
          'x': 3,
          'ß': 1}),
 Counter({'t': 70355,
          'h': 48738,
          'e': 93720,
          'p': 13417,
          'r

Trying to find common letters and their probs

In [38]:
common=set(texts[0]) & set(texts[1]) & set(texts[2]) & set(texts[3])  

def probs(text):
    total_letter=len(text)
    each_letter=Counter(text)
    return ([each_letter[letter]/total_letter for letter in common]) 

all_probs=[probs(text) for text in texts]

In order to determine unknown text's language, KL divergence can be applied to find the similarities between languages

In [56]:
from scipy.stats import entropy
import pandas as pd
name_of_texts=['Nutuk','Great Expectations','Madame Bovary','Unknown']

dists=np.zeros((4,4))
for i in range(len(texts)):
    for j in range(len(texts)):
        dists[i][j]=entropy(all_probs[i],all_probs[j])
        
pd.DataFrame(dists,index=name_of_texts,columns=name_of_texts)

,Nutuk,Great Expectations,Madame Bovary,Unknown
Nutuk,0.000000,0.229013,0.365576,0.377345
Great Expectations,0.358755,0.000000,0.234775,0.043879
Madame Bovary,0.289477,0.146872,0.000000,0.185256
Unknown,0.430059,0.038013,0.237447,0.000000


The min distance of unknown text is between english. So it is probably in english

## Question #2

For this question consider the [Car Evaluation Data Set](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation) from UCI. Here is the [direct link](https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data) to the dataset.

Make [contingency tables](https://en.wikipedia.org/wiki/Contingency_table#:~:text=In%20statistics%2C%20a%20contingency%20table,%2C%20engineering%2C%20and%20scientific%20research.) of the columns (using [`crosstab`](https://pandas.pydata.org/docs/reference/api/pandas.crosstab.html) function from [pandas](https://pandas.pydata.org)) and figure out which pairs of columns are dependent and independent. Explain your result using statistical tests.

In [47]:
import pandas as pd
import numpy as np

data = pd.read_excel('https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls', header=None)
data.head()

ct=np.zeros((7,7))

for i in range(6):
    for j in range(i+1,7):
        x=pd.crosstab(data[i],data[j])
        chi=chisquare(x,axis=None)
        ct[i][j]=chi.statistic
        ct[j][i]=ct[i][j]
        
columns = ['class','buying','maint','doors','persons','log_boot','safety']

pd.DataFrame(ct,index=columns,columns=columns)

        

,class,buying,maint,doors,persons,log_boot,safety
class,0.0,2.430081e+06,60002.000000,210007.000000,120004.000000,1.680056e+06,330011.000000
buying,2430081.0,0.000000e+00,174808.337444,319091.153123,260352.748617,1.878247e+05,415566.522099
maint,60002.0,1.748083e+05,0.000000,178987.222585,154165.413372,9.281088e+04,220306.702353
doors,210007.0,3.190912e+05,178987.222585,0.000000,272787.123258,1.777746e+05,385824.969402
persons,120004.0,2.603527e+05,154165.413372,272787.123258,0.000000,1.852219e+05,326660.703286
log_boot,1680056.0,1.878247e+05,92810.875408,177774.600227,185221.930138,0.000000e+00,208450.336644
safety,330011.0,4.155665e+05,220306.702353,385824.969402,326660.703286,2.084503e+05,0.000000


Safety is related with each column

## Question #3

For this question, use [Default of Credit Card Clients Data Set]() from UCI. Here is the [direct link](https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls) to the dataset.

Your tasks are

* Inspect the dataset.
* Would it be appropriate to form a linear regression model to predict the `default payment next month` variable? Explain.
* Form a [contingency table](https://en.wikipedia.org/wiki/Contingency_table#:~:text=In%20statistics%2C%20a%20contingency%20table,%2C%20engineering%2C%20and%20scientific%20research.) of the columns `SEX` vs `default payment next month` and `EDUCATION` vs `default payment next month`.
* Are there statistically verifiable relationships between credit card defaults, the gender of and the education level the borrower? Which is stronger? Quantify your analysis using [Chi Square Test](https://en.wikipedia.org/wiki/Chi-squared_test).

In [49]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
data=pd.read_excel("https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls",header=1)


In [50]:
data.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


 The target column (default payment next month) consists of binary numbers, so this is not a linear regression problem. 

In [59]:
a=pd.crosstab(data['SEX'],data['default payment next month'],normalize=0)
b=pd.crosstab(data['EDUCATION'],data['default payment next month'],normalize=0)
sexvs=chisquare(a,axis=None)
educvs=chisquare(b,axis=None)


In [62]:
a

default payment next month,0,1
SEX,,
1,0.758328,0.241672
2,0.792237,0.207763


In [63]:
b

default payment next month,0,1
EDUCATION,,
0,1.000000,0.000000
1,0.807652,0.192348
2,0.762651,0.237349
3,0.748424,0.251576
4,0.943089,0.056911
5,0.935714,0.064286
6,0.843137,0.156863


In [60]:
sexvs

Power_divergenceResult(statistic=0.6085431576951181, pvalue=0.8944738579011734)

In [61]:
educvs

Power_divergenceResult(statistic=3.9170740465319063, pvalue=0.992041909941499)

p values are close to 1 so there isn't any correlation between sex, education and payment

#### P values are 0 so these features has a relationship between default payment and the test value of education is higher, so that means education is more related to payment

## Question #4

For this question, use the [Iris Dataset](https://archive.ics.uci.edu/ml/datasets/iris) from UCI.  Here is the [direct link](https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data) to the dataset.

Your tasks are

* Form a [K-NN](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) model for this dataset.
* Test your model on random samples of your data and calculate its accuracy.
* Repeat your calculation 100 times and give an interval of accuracy values leaving the best 2.5% and worst 2.5% accuracy values.
* Is there a better way of doing this without repeating the calculation 100 times? Explain.
* Find the best parameter $k$ for your dataset for the K-NN model.

In [65]:
data=pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data",header=None)

In [75]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import ShuffleSplit

X=data.iloc[:,0:4]
y=data[4]
scores=[]

for i in range(100):
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)
    model2=KNeighborsClassifier(n_neighbors=3)
    
    model2.fit(X_train,y_train)
    y_preds=model2.predict(X_test)
    sc=accuracy_score(y_test,y_preds)
    
    scores.append(sc)
    
scores.sort()
results=scores[2:98]
sum(results)/len(results)

0.959722222222223

T-test can be used instead of 100 experiments

In [88]:
from scipy.stats import t
scores=[]
for i in range(10):
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25)
    model2=KNeighborsClassifier(n_neighbors=3)
    
    model2.fit(X_train,y_train)
    y_preds=model2.predict(X_test)
    sc=accuracy_score(y_test,y_preds)
    
    scores.append(sc)
    
u=np.mean(scores)
sigma=np.std(scores)

t.interval(0.95,9,loc=u,scale=sigma)


(0.8652886419272108, 0.9978692528096313)

confidence interval is found between 0.86 and 0.99 with only 10 experiment

In [97]:
ks=[1,2,3,5,10,25,50,100]

scoresofk=[]
for k in ks:
    scores=[]
    for i in range(10):
        
        X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25)
        model4=KNeighborsClassifier(n_neighbors=k)

        model4.fit(X_train,y_train)
        y_preds=model4.predict(X_test)
        sc=accuracy_score(y_test,y_preds)

        scores.append(sc)
    temp=np.mean(scores)
    scoresofk.append(temp)


In [98]:
scoresofk

[0.9578947368421054,
 0.9315789473684211,
 0.95,
 0.9657894736842104,
 0.9631578947368421,
 0.9526315789473683,
 0.9473684210526315,
 0.4184210526315789]

In [100]:
import numpy as np
print("k ={} is the best k-nn".format(ks[np.argmax(scoresofk)]))

k =5 is the best k-nn


## Question #5

For this question, we are going to use [Concrete Slump Test Dataset](https://archive.ics.uci.edu/ml/datasets/Concrete+Slump+Test) from UCI. Here is the [direct link](https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/slump/slump_test.data) to the dataset.

Your tasks are

* Form three separate linear regression model for the following dependent variables:

  - SLUMP (cm)
  - FLOW (cm)
  - 28-day Compressive Strength (Mpa)
  
* Compare how well these models fit.

In [102]:
data=pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/concrete/slump/slump_test.data")
data

,No,Cement,Slag,Fly ash,Water,SP,Coarse Aggr.,Fine Aggr.,SLUMP(cm),FLOW(cm),Compressive Strength (28-day)(Mpa)
0,1,273.0,82.0,105.0,210.0,9.0,904.0,680.0,23.0,62.0,34.99
1,2,163.0,149.0,191.0,180.0,12.0,843.0,746.0,0.0,20.0,41.14
2,3,162.0,148.0,191.0,179.0,16.0,840.0,743.0,1.0,20.0,41.81
3,4,162.0,148.0,190.0,179.0,19.0,838.0,741.0,3.0,21.5,42.08
4,5,154.0,112.0,144.0,220.0,10.0,923.0,658.0,20.0,64.0,26.82
...,...,...,...,...,...,...,...,...,...,...,...
98,99,248.3,101.0,239.1,168.9,7.7,954.2,640.6,0.0,20.0,49.97
99,100,248.0,101.0,239.9,169.1,7.7,949.9,644.1,2.0,20.0,50.23
100,101,258.8,88.0,239.6,175.3,7.6,938.9,646.0,0.0,20.0,50.50
101,102,297.1,40.9,239.9,194.0,7.5,908.9,651.8,27.5,67.0,49.17


In [103]:
X0=data.iloc[:,10]
slump=data.iloc[:,8]
flow=data.iloc[:,9]
X=data.iloc[:,1:8]

In [104]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

def linearreg(X,y):
    model=LinearRegression()
    model.fit(X,y)
    return model.score(X,y)
a=linearreg(X,X0)
b=linearreg(X,slump)
c=linearreg(X,flow)



In [106]:
a,b,c

(0.8968376098140092, 0.3233078797083553, 0.5022296725445197)

Compressive strength has a relation with the rest of data. So when models are created and model scores are obtained, R^2 of it  is higher then the others, other models couldnt fit well so we can say that they are independent variables.